In [86]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
%matplotlib inline 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

import requests
import kdtree as KDTree


In [87]:
df = pd.read_csv('houses_lat_long_v4.csv')

In [88]:
df = df.dropna(subset=['LAT_LONG'])

In [89]:
df.shape

(87158, 31)

In [90]:
# First, look for null values so we can dropna
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

LAND_SF          19
YR_REMOD       7329
TOTAL_RMS        33
BTH_STYLE2    23961
BTH_STYLE3    58492
dtype: int64

In [119]:
# Get the coords
def split_coords(row):
    x = row['LAT_LONG'].split(",")
    y = (x[0], x[1])
    return y[0], y[1]


# A function that returns multiple things.
def some_func(x):
    x = co.split(",")
    y = (x[0], x[1])
    return x[0], x[1]


# Example usage.
df['B'], df['C'] = zip(*df['LAT_LONG'].apply(some_func))

NameError: name 'row' is not defined

In [118]:
df

,PID,ST_NUM,ST_NAME,ST_NAME_SUF,ZIPCODE,OWN_OCC,AV_LAND,AV_BLDG,AV_TOTAL,GROSS_TAX,...,AC,FPLACE,INT_CND,INT_FIN,VIEW,LAT_LONG,LAT,LONG,B,C
0,502555000,2,BEAVER,ST,2108,Y,1436300,1242500,2678800,2823455,...,N,0.0,G,N,A,"-71.07207,42.355618",0,1,"-71.07207,42.355618","-71.07207,42.355618"
1,100001000,104 A 104,PUTNAM,ST,2128,Y,129900,411300,541200,570425,...,N,0.0,A,N,A,"-71.03256,42.379173",0,1,"-71.03256,42.379173","-71.03256,42.379173"
2,100002000,197,LEXINGTON,ST,2128,N,121000,475100,596100,628289,...,C,0.0,A,N,A,"-71.03255,42.37957",0,1,"-71.03255,42.37957","-71.03255,42.37957"
3,100003000,199,LEXINGTON,ST,2128,N,121400,421100,542500,571795,...,N,0.0,A,N,A,"-71.03248,42.379593",0,1,"-71.03248,42.379593","-71.03248,42.379593"
4,100004000,201,LEXINGTON,ST,2128,N,121900,396400,518300,546288,...,N,0.0,A,N,A,"-71.03242,42.379616",0,1,"-71.03242,42.379616","-71.03242,42.379616"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,100135000,432,SARATOGA,ST,2128,Y,175000,426900,601900,634403,...,N,0.0,A,N,A,"-71.02874,42.379467",0,1,"-71.02874,42.379467","-71.02874,42.379467"
104,100136000,428,SARATOGA,ST,2128,N,158200,430600,588800,620595,...,N,0.0,A,N,A,"-71.02886,42.37942",0,1,"-71.02886,42.37942","-71.02886,42.37942"
105,100137000,426-R,SARATOGA,ST,2128,N,105900,298500,404400,426238,...,N,0.0,G,N,A,"-71.02892,42.3794",0,1,"-71.02892,42.3794","-71.02892,42.3794"
106,100138000,426,SARATOGA,ST,2128,N,112700,380700,493400,520044,...,N,0.0,G,N,A,"-71.02892,42.3794",0,1,"-71.02892,42.3794","-71.02892,42.3794"


In [117]:
df['LAT'], df['LONG'] = df.apply(split_coords, axis=1)

ValueError: too many values to unpack (expected 2)

In [110]:
df

,PID,ST_NUM,ST_NAME,ST_NAME_SUF,ZIPCODE,OWN_OCC,AV_LAND,AV_BLDG,AV_TOTAL,GROSS_TAX,...,KITCH_STYLE,HEAT_TYP,AC,FPLACE,INT_CND,INT_FIN,VIEW,LAT_LONG,LAT,LONG
0,502555000,2,BEAVER,ST,2108,Y,1436300,1242500,2678800,2823455,...,M,W,N,0.0,G,N,A,"-71.07207,42.355618",0,1
1,100001000,104 A 104,PUTNAM,ST,2128,Y,129900,411300,541200,570425,...,S,W,N,0.0,A,N,A,"-71.03256,42.379173",0,1
2,100002000,197,LEXINGTON,ST,2128,N,121000,475100,596100,628289,...,M,F,C,0.0,A,N,A,"-71.03255,42.37957",0,1
3,100003000,199,LEXINGTON,ST,2128,N,121400,421100,542500,571795,...,S,S,N,0.0,A,N,A,"-71.03248,42.379593",0,1
4,100004000,201,LEXINGTON,ST,2128,N,121900,396400,518300,546288,...,S,W,N,0.0,A,N,A,"-71.03242,42.379616",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,100135000,432,SARATOGA,ST,2128,Y,175000,426900,601900,634403,...,S,W,N,0.0,A,N,A,"-71.02874,42.379467",0,1
104,100136000,428,SARATOGA,ST,2128,N,158200,430600,588800,620595,...,S,W,N,0.0,A,N,A,"-71.02886,42.37942",0,1
105,100137000,426-R,SARATOGA,ST,2128,N,105900,298500,404400,426238,...,M,E,N,0.0,G,N,A,"-71.02892,42.3794",0,1
106,100138000,426,SARATOGA,ST,2128,N,112700,380700,493400,520044,...,M,E,N,0.0,G,N,A,"-71.02892,42.3794",0,1


In [ ]:
df = df.fillna(0)

In [ ]:
# features: bed, bath, sqft, lon/lat, garage, age (or year built), and lotsize

features = ['BDRMS', 'FULL_BTH', 'HALF_BTH', 'LIVING_AREA', 'YR_BUILT', 'LAND_SF', 'LAT', 'LONG']
X = df[features] #Features
y = df['AV_TOTAL'] #Target
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)

In [ ]:
knn.predict(X_test)

In [ ]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors([[-5,-5]])
print(indices)
print(distances)

In [ ]:
for d in distances:
    weight = 1.0/d
print(weight)